In [1]:
import pandas as pd
import os
import pyodbc
import sqlalchemy


In [2]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server']

In [17]:
from dotenv import load_dotenv

load_dotenv()

connection_string = os.getenv('DB_CONNECTION_STRING')
conn = pyodbc.connect(connection_string)
print("Connection Successful")


Connection Successful


In [ ]:
import csv

# Get connection from environment variable
connection_string = os.getenv('DB_CONNECTION_STRING')
connection = pyodbc.connect(connection_string)

try:
    with open('DataSources/orders.csv', 'r') as f:
        reader = csv.reader(f)
        columns = next(reader)
        
        
        # Clean column names for SQL
        clean_columns = [c.strip().replace(' ', '_') for c in columns]
        
        # Specify database and table
        table_name = "PythonETL.dbo.orders"  # Using the name from your error message
        
        # Create table if it doesn't exist
        # Assuming all columns are VARCHAR(255) for simplicity
        create_query = f"IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'orders' AND schema_id = SCHEMA_ID('dbo') AND OBJECT_ID IN (SELECT OBJECT_ID FROM sys.objects WHERE type = 'U' AND DB_NAME() = 'PythonETL')) BEGIN CREATE TABLE {table_name} ({', '.join([f'{col} VARCHAR(255)' for col in clean_columns])}) END"
        
        cursor = connection.cursor()
        cursor.execute(create_query)
        connection.commit()
        
        # Now insert the data
        insert_query = 'INSERT INTO {0}({1}) VALUES ({2})'
        insert_query = insert_query.format(
            table_name,
            ','.join(clean_columns),
            ','.join('?' * len(columns))
        )
        
        count = 0
        for data in reader:
            cursor.execute(insert_query, data)
            count += 1
            
            if count % 1000 == 0:
                connection.commit()
                print(f"Inserted {count} rows...")
        
        connection.commit()
        print(f"Successfully inserted {count} rows into {table_name}")
        
except Exception as e:
    print(f"Error: {str(e)}")
    connection.rollback()
finally:
    cursor.close()
    connection.close()

Inserted 1000 rows...
Successfully inserted 1615 rows into PythonETL.dbo.orders


In [39]:
query = """
SELECT *
FROM PythonETL.dbo.orders
"""

orders = pd.read_sql(query, conn)
orders.head()




C:\Users\oo\AppData\Local\Temp\ipykernel_4464\3171140347.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders = pd.read_sql(query, conn)


,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6


In [6]:
conn.close()